**Frame generation**

In [ ]:
#Libraries and functions

import cv2
import numpy as np
import glob # To find files that have an specific pattern using a wildcard
import os

##############################VIDEO TO BE PROCESSED##############################
#video_path          = '/kaggle/input/workzone/workzone2.mp4' #/kaggle/input/d/justinfb/workzone/workzone2.mp4
video_path          = '/kaggle/input/d/justinfb/workzone/workzone2.mp4'
#video_path          = '/kaggle/input/workzone6-1-25/CV_Work Zone/workzone.mp4'
video_cv2           = cv2.VideoCapture(video_path)
fps_cv2             = video_cv2.get(cv2.CAP_PROP_FPS)
# TO-ASK: Why these values are hard-coded?
video_initial_video = 3 * 60 + 48 # 3 minutes and 48 seconds.
duration_video      = 45          # 45 seconds
##############################VIDEO TO BE PROCESSED##############################

output_folder = '/kaggle/working/frames'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def video_to_images(video_path, output_folder):
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    
    # Check if the output folder exists, if not, create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    frame_count = 0
    
    # Read video frame by frame and save each frame
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        
        # Create the filename with the frame number
        filename = os.path.join(output_folder, f"frame_{frame_count:04d}.png")
        
        # Save the frame as an image
        cv2.imwrite(filename, frame)
        
        frame_count += 1
    
    # Release the video capture object
    video_capture.release()
    
    print(f"Processed and saved {frame_count} frames to {output_folder}")

# Usage
video_to_images(video_path, output_folder)

**Vision transformer**

In [ ]:
!git clone https://github.com/isl-org/DPT.git

# Download models and weights
!wget https://github.com/intel-isl/DPT/releases/download/1_0/dpt_hybrid-midas-501f0c75.pt
!wget https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt
!wget https://github.com/intel-isl/DPT/releases/download/1_0/dpt_hybrid-ade20k-53898607.pt
!wget https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-ade20k-b12dca68.pt
    
# Import weights
!mv ./dpt_hybrid-ade20k-53898607.pt ./DPT/weights
!mv ./dpt_large-ade20k-b12dca68.pt ./DPT/weights
!mv ./dpt_large-midas-2f21e586.pt ./DPT/weights
!mv ./dpt_hybrid-midas-501f0c75.pt ./DPT/weights

# Pip install required libraries with last releases
!pip install torch
!pip install torchvision
!pip install opencv-python
!pip install timm

In [ ]:
import shutil # To make actions in an easy way in the directories and files
import os
import cv2

def count_frames(cuts,video_path):
    """
    Converts time-based video cuts (HH:MM:SS) into frame indices based on the video's FPS.
    
    Parameters:
        cuts (list of tuples): List of (start_time, end_time) tuples in "HH:MM:SS" format.
        video_path (str): Path to the video file.

    Returns:
        list: A list of start and end frame indices for each cut.
    """
    # Empty list with frames
    video  = cv2.VideoCapture(video_path)
    fps    = video.get(cv2.CAP_PROP_FPS)
    frames = []
    
    for idx, (start_time, duration) in enumerate(cuts):
        # Start frame
        start_seconds = int(cuts[idx][0].split(":")[2])
        start_minutes = int(cuts[idx][0].split(":")[1])
        start_hours   = int(cuts[idx][0].split(":")[0])
        frame_start   = int((start_seconds + start_minutes * 60 + start_hours * 60 * 60) * fps)
        frames.append(frame_start)

        end_seconds = int(cuts[idx][1].split(":")[2])
        end_minutes = int(cuts[idx][1].split(":")[1])
        end_hours   = int(cuts[idx][1].split(":")[0])
        frame_end   = int((end_seconds + end_minutes * 60 + end_hours * 60 * 60) * fps)
        frames.append(frame_end)
        
    return frames

def count_all_frames_from_frames(dir_frame):
    frame_list       = os.listdir(dir_frame)
    folder_name      = dir_frame.split('/')[-1]
    frames_range     = [0]
    frames_range.append(len(frame_list))
    return frames_range

def count_all_frames(dir_frame):
    frame_list       = os.listdir(dir_frame)
    folder_name      = dir_frame.split('/')[-1]
    frames_range     = [0]
    frames_range.append(len(frame_list))
    return frames_range

def get_some_frames(dir_frame, offset, duration):
    frames_range     = [offset]
    frames_range.append(offset + duration)
    return frames_range

def rename_files_for_processing(cuts,video_path,frame_dir):
    # Empty list with frames
    video  = cv2.VideoCapture(video_path)
    fps    = video.get(cv2.CAP_PROP_FPS)
    #frames = count_frames(cuts,video_path)
    frames = count_all_frames_from_frames(frame_dir)
        
    #print(frames)
    #print(fps)
    #video_name = video_path.split('/')[-1]
    #frame_name = frame_dir + video_name.split('.avi')[0]
    frame_name = frame_dir + '/' + frame_dir.split('/')[-1]
    # It goes through the required frames, it is divided by 2 because it comes in pairs
    for frames_sample in range(int(len(frames)/2)):
        print('Processing frame ' + frame_name + ' of frame pair ' + str(frames[0 + frames_sample * 2]) + ' to ' + str(frames[1 + frames_sample * 2]))
        #It goes from start frame to the last of that pair
        for copy_frames in range(frames[1  + frames_sample * 2] - frames[0 + frames_sample * 2]):
            #To check names correctly choose
            #print(frame_name + str(copy_frames + frames[0]) + '.jpg')
            # To get the name of the new picture
            file_name_from_frame = frame_name.split('/')[-1]
            # The copy_frames + frames[0] gives the resulting frame to be processed
            shutil.copyfile(frame_name + str(copy_frames + frames[0 + frames_sample * 2]) + '.jpg',
                            '/kaggle/working/DPT/input/' + file_name_from_frame + str(copy_frames + frames[0 + frames_sample * 2]) + '.jpg')
            
def rename_files_for_processing_custom_cuts(offset, duration,frame_dir):
    # Empty list with frames
    #video  = cv2.VideoCapture(video_path)
    #fps    = video.get(cv2.CAP_PROP_FPS)
    #frames = count_frames(cuts,video_path)
    #frames = count_all_frames_from_frames(frame_dir)
    frames = get_some_frames(frame_dir, offset, duration)
    
    #print(frames)
    #print(fps)
    #video_name = video_path.split('/')[-1]
    #frame_name = frame_dir + video_name.split('.avi')[0]
    #frame_name = frame_dir + '/' + frame_dir.split('/')[4]
    #print(frame_dir.split('/')[4])
    print('Processing frame from ' + str(frames[0]) + ' to ' + str(frames[1]))
    for copy_frames in range(frames[1] - frames[0]):
        #To check names correctly choose
        #print(frame_name + str(copy_frames + frames[0]) + '.jpg')
        # To get the name of the new picture
        #file_name_from_frame = frame_name.split('/')[-1]
        #print(file_name_from_frame)
        #print(frame_name)
        #print(str(copy_frames + frames[0]))
        # The copy_frames + frames[0] gives the resulting frame to be processed
        if ((copy_frames + frames[0]) < 10):
            zeros_added = "000"
        elif ((copy_frames + frames[0]) < 100):
            zeros_added = "00"
        elif ((copy_frames + frames[0]) < 1000):
            zeros_added = "0"
        else:
            zeros_added = ""
        shutil.copyfile("/kaggle/working/frames/frame_" + zeros_added + str(copy_frames + frames[0]) + '.png',
                        '/kaggle/working/DPT/input/' + "frame_" + zeros_added + str(copy_frames + frames[0]) + '.png')
        
        #shutil.copyfile(frame_name + str(copy_frames + frames[0]) + '.jpg',
        #                '/kaggle/working/DPT/input/' + str(copy_frames + frames[0]) + '.jpg')
            
def get_all_files(cuts,video_dir,frame_dir):
    video_list = os.listdir(video_dir)
    for video_path in video_list:
        rename_files_for_processing(cuts,video_dir + video_path,frame_dir)
        #rename_files_for_processing_custom_cuts(offset, duration,video_dir + video_path,frame_dir)
        ################################################
        break
        
def get_all_files_custom(offset, duration,frame_dir):
    #print(frame_dir)
    rename_files_for_processing_custom_cuts(offset, duration, frame_dir)
        
def initialize_system():
    # Generate output directory
    if(not(os.path.isdir('/kaggle/working/output'))):
        os.mkdir('/kaggle/working/output')
    
    filename = "/kaggle/working/DPT/run_monodepth.py"
    text = open(filename).read()
    open(filename, "w+").write(text.replace('"output_monodepth"', '"/kaggle/working/DPT/output_monodepth"'))

    filename = "/kaggle/working/DPT/run_segmentation.py"
    text = open(filename).read()
    open(filename, "w+").write(text.replace('"output_semseg"', '"/kaggle/working/DPT/output_semseg"'))
    
    filename = "/kaggle/working/DPT/run_monodepth.py"
    text = open(filename).read()

    #Here goes your files
    open(filename, "w+").write(text.replace('"input"', '"/kaggle/working/DPT/input/"'))

    filename = "/kaggle/working/DPT/run_segmentation.py"
    text = open(filename).read()

    #Here goes your files
    open(filename, "w+").write(text.replace('"input"', '"/kaggle/working/DPT/input/"'))
    
    filename = "/kaggle/working/DPT/run_monodepth.py"
    text = open(filename).read()
    open(filename, "w+").write(text.replace('"weights/', '"/kaggle/working/DPT/weights/'))

    filename = "/kaggle/working/DPT/run_segmentation.py"
    text = open(filename).read()
    open(filename, "w+").write(text.replace('"weights/', '"/kaggle/working/DPT/weights/'))
    
def copy_files_to_output_depth(video_dir,cuts):
    output_file_depth = '/kaggle/working/DPT/output_monodepth/'
    output_file_segm  = '/kaggle/working/DPT/output_semseg/'
    output_file       = '/kaggle/working/output/depth'
    
    video_list = os.listdir(video_dir)
    
    # Generate output directory
    if(not(os.path.isdir(output_file))):
        os.mkdir(output_file)

    #Create zip files to extract them
    # This for goes through all the video directories
    for sample_picture_0 in video_list:
        # We know that every video ends in 0.jpg, where 0 is the frame, so we must build each one
        frame_name = sample_picture_0.split('.avi')[0]
        frames     = count_frames(cuts,video_dir + sample_picture_0)
        # It goes through the required frames, it is divided by 2 because it comes in pairs
        for frames_sample in range(int(len(frames)/2)):
            print('Processing frame ' + sample_picture_0 + ' of frame pair ' + str(frames_sample))
            #It goes from start frame to the last of that pair
            for copy_frames in range(frames[1 + frames_sample * 2] - frames[0 + frames_sample * 2]):
                #To check names correctly choose
                #print(frame_name + str(copy_frames + frames[0]) + '.jpg')

                # The copy_frames + frames[0] gives the resulting frame to be processed for depth, png and pfm
                shutil.move(output_file_depth + frame_name + str(copy_frames + frames[0 + frames_sample * 2]) + '.png',
                            output_file + frame_name + str(copy_frames + frames[0 + frames_sample * 2]) + '.png')
                shutil.move(output_file_depth + frame_name + str(copy_frames + frames[0 + frames_sample * 2]) + '.pfm',
                            output_file + frame_name + str(copy_frames + frames[0 + frames_sample * 2]) + '.pfm')

def copy_files_to_output_segmentation(video_dir,cuts):
    output_file_depth = '/kaggle/working/DPT/output_monodepth/'
    output_file_segm  = '/kaggle/working/DPT/output_semseg/'
    output_file       = '/kaggle/working/output/segmentation'
    
    video_list = os.listdir(video_dir)
    
    if(not(os.path.isdir(output_file))):
        os.mkdir(output_file)

    #Create zip files to extract them
    # This for goes through all the video directories
    for sample_picture_0 in video_list:
        # We know that every video ends in 0.jpg, where 0 is the frame, so we must build each one
        frame_name = sample_picture_0.split('.avi')[0]
        frames     = count_frames(cuts,video_dir + sample_picture_0)
        # It goes through the required frames, it is divided by 2 because it comes in pairs
        for frames_sample in range(int(len(frames)/2)):
            print('Processing frame ' + sample_picture_0 + ' of frame pair ' + str(frames_sample))
            #It goes from start frame to the last of that pair
            for copy_frames in range(frames[1 + frames_sample * 2] - frames[0 + frames_sample * 2]):
                #To check names correctly choose
                #print(frame_name + str(copy_frames + frames[0]) + '.jpg')

                # The copy_frames + frames[0] gives the resulting frame to be processed for segmentation
                shutil.move(output_file_segm + frame_name + str(copy_frames + frames[0 + frames_sample * 2]) + '.png',
                            output_file + frame_name + str(copy_frames + frames[0 + frames_sample * 2]) + '.png')

def copy_output_monodepth_from_dir(frame_dir_video, offset, duration):
    output_file_depth = '/kaggle/working/DPT/output_monodepth/'
    #output_file_segm  = '/kaggle/working/DPT/output_semseg/'
    input_file        = '/kaggle/working/DPT/input/'
    frame_name        = frame_dir_video.split('/')[-1]
    output_file       = '/kaggle/working/output/' + frame_name + '/monocular/'
    #output_file       = '/kaggle/working/output/segmentation'
    
    list_files = os.listdir(output_file_depth)
    frames     = count_all_frames_from_frames(frame_dir_video)
    
    #if(not(os.path.isdir(output_file))):
    #    os.mkdir(output_file)
    
    os.makedirs(output_file, exist_ok=True)
    
    for studied_frame in list_files:
        if studied_frame.endswith('.png'):
            shutil.move(output_file_depth + studied_frame,output_file + studied_frame)
            #/kaggle/working/DPT/input/soads_2024-06-26-16-40-11-_front_camera_image_compressed378.jpg
            # Remove already process frame
            os.remove(input_file + studied_frame.replace('.png','.png'))
        else:
            os.remove(output_file_depth + studied_frame)
        
def copy_outputs_segmentation_from_dir(frame_dir_video, offset, duration):
    #output_file_depth = '/kaggle/working/DPT/output_monodepth/'
    output_file_segm  = '/kaggle/working/DPT/output_semseg/'
    input_file        = '/kaggle/working/DPT/input/'
    frame_name        = frame_dir_video.split('/')[-1]
    output_file       = '/kaggle/working/output/' + frame_name + '/segmentation/'
    #output_file       = '/kaggle/working/output/segmentation'
    
    list_files = os.listdir(output_file_segm)
    #frames     = get_some_frames(dir_frame, offset, duration)
    frames     = count_all_frames_from_frames(frame_dir_video)
    
    #if(not(os.path.isdir(output_file))):
    #    os.mkdir(output_file)
    
    os.makedirs(output_file, exist_ok=True)
    
    for studied_frame in list_files:
        if studied_frame.endswith('.png'):
            shutil.move(output_file_segm + studied_frame, output_file + studied_frame)
            os.remove(input_file + studied_frame.replace('.png','.png'))
        else:
            os.remove(output_file_segm + studied_frame)
        
def execute_monodepth(video_dir,cuts):
    !python /kaggle/working/DPT/run_monodepth.py
    copy_files_to_output_depth(video_dir,cuts)
    
def execute_monodepth_dir(video_dir, offset, duration):
    !python /kaggle/working/DPT/run_monodepth.py
    copy_output_monodepth_from_dir(video_dir, offset, duration)

def execute_segmentation(video_dir,cuts):
    !python /kaggle/working/DPT/run_segmentation.py
    copy_files_to_output_segmentation(video_dir,cuts)
    
def execute_segmentation_dir(video_dir, offset, duration):
    !python /kaggle/working/DPT/run_segmentation.py
    copy_outputs_segmentation_from_dir(video_dir, offset, duration) 

In [ ]:
frame_dirs      = output_folder
list_dir_frames = os.listdir(frame_dirs)

#print(list_dir_frames)

#max_frames      = count_all_frames_from_frames(frame_dirs + list_dir_frames[video_num])[1]

max_frames      = count_all_frames(frame_dirs)[1]
#max_frames      = (video_initial_video + duration_video) * fps_cv2
#offset          = video_initial_video * fps_cv2
#print(max_frames)
#break


frame_counter   = max_frames
#frame_counter   = 5200
#frame_counter   = 1200
print('Monodepth: Maximum frames are: ' + str(max_frames))
frame_segm      = 100
offset          = 0
initial_offset  = offset

while True:
    get_all_files_custom(offset, frame_segm, frame_dirs)
    initialize_system()
    execute_monodepth_dir(frame_dirs, offset, frame_segm)
    offset        += frame_segm
    #frame_counter -= frame_segm
    if(offset + frame_segm >= max_frames) or (offset >= frame_counter):
        break

# If there are some frames left
if(offset + frame_segm > max_frames):
    frame_counter_diff = max_frames - offset
    print(frame_counter_diff)
    get_all_files_custom(offset, frame_counter_diff, frame_dirs)
    initialize_system()
    #Monodepth
    execute_monodepth_dir(frame_dirs, offset, frame_counter_diff)
    
print('Segmentation: Maximum frames are: ' + str(max_frames))
frame_segm      = 100
offset          = 0
initial_offset  = offset

while True:
    get_all_files_custom(offset, frame_segm, frame_dirs)
    initialize_system()
    #Segmentation
    execute_segmentation_dir(frame_dirs, offset, frame_segm)
    offset        += frame_segm
    #frame_counter -= frame_segm
    if(offset + frame_segm >= max_frames) or (offset >= frame_counter):
        break

# If there are some frames left
if(offset + frame_segm > max_frames):
    frame_counter_diff = max_frames - offset
    print(frame_counter_diff)
    get_all_files_custom(offset, frame_counter_diff, frame_dirs)
    initialize_system()
    #Segmentation
    execute_segmentation_dir(frame_dirs, offset, frame_counter_diff)

**Map generation**

In [ ]:
!pip install open3d

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import imageio.v3 as iio
import matplotlib.pyplot as plt
import open3d as o3d
import os
import math
import cv2
import copy
from PIL import Image
from numpy import asarray

In [ ]:
def increase_brightness(image_path, value):
    # Read the image
    image = cv2.imread(image_path)
    # Check if image is loaded successfully
    if image is None:
        raise ValueError(f"Image at path '{image_path}' could not be loaded.")
    # Convert the image to HSV (Hue, Saturation, Value) color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Split the image into three channels: H, S, and V
    h, s, v = cv2.split(hsv)
    # Increase the V (value/brightness) channel by the given value
    v = cv2.add(v, value)
    # Merge the channels back
    final_hsv = cv2.merge((h, s, v))
    # Convert the HSV image back to BGR color space
    brightened_image = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return brightened_image

#This is for Vision Transformers because it gives an inverse relation of white and black is.
def normalize_image_to_pcd(input_path,brightness_value):
    brightness_image  = increase_brightness(input_path,brightness_value)
    cv2.imwrite('brightened_image.jpg', brightness_image)
    input_image       = Image.open('brightened_image.jpg').convert('L')
    input_image_array = input_image.resize((640,480))
    # It reverse depth making black, white and viceversa.
    #max_pixel_value   = input_image_array
    reverse_depth     = 255 - asarray(input_image_array)
    #print(len(reverse_depth))
    return(reverse_depth)

#It extracts color from image.
def extract_color_image(img_path):
    img = Image.open(img_path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    if img.size != (640,480):
        img = img.resize((640, 480))
    img = asarray(img)
    img = img/255.0 # normalize RGB values to [0, 1]
    height, width, channels = img.shape
    length = height * width

    color_list = img.reshape((length, channels)) # array of RGB values
    return color_list

#It creates the PCD from depth.
def depth_to_pcd(depth_path, color_image_path, brightness_value):
    # Depth camera parameters from NYU:
    FX_DEPTH = 5.8262448167737955e+02
    FY_DEPTH = 5.8269103270988637e+02
    CX_DEPTH = 3.1304475870804731e+02
    CY_DEPTH = 2.3844389626620386e+02
    
    # It only has one channel it is in grey scale, it change size and invert greyscale
    depth_image = normalize_image_to_pcd(depth_path,brightness_value)
    
    # get depth resolution:
    height, width = depth_image.shape
    length = height * width
    # compute indices:
    jj = np.tile(range(width), height)
    ii = np.repeat(range(height), width)
    # rechape depth image
    z = depth_image.reshape(length)
    # compute pcd:
    pcd = np.dstack([(ii - CX_DEPTH) * z / FX_DEPTH,
                 (jj - CY_DEPTH) * z / FY_DEPTH,
                 z]).reshape((length, 3))

    pcd_o3d = o3d.geometry.PointCloud()
    pcd_o3d.points = o3d.utility.Vector3dVector(pcd)
    colors = extract_color_image(color_image_path)
    pcd_o3d.colors = o3d.utility.Vector3dVector(colors)
    # Define the box parameters
    center         = np.array([0.0, 0.0, 225])
    # Consider 70
    extent         = np.array([400, 400, 100])
    rotation       = np.eye(3)  # Identity matrix for no rotation
    # Cut the points inside the box, eliminate point in infinite or far far away
    pcd_o3d        = cut_points_in_box(pcd_o3d, center, extent, rotation)
    #o3d.io.write_point_cloud(output_pcd_path, pcd_o3d)
    return pcd_o3d

def cut_points_in_box(point_cloud, box_center, box_extent, box_rotation=None):
    """
    Cut points inside a box from the point cloud.
    
    Parameters:
    - point_cloud: open3d.geometry.PointCloud, the input point cloud.
    - box_center: list or np.array of shape (3,), the center of the box.
    - box_extent: list or np.array of shape (3,), the size of the box (x, y, z dimensions).
    - box_rotation: optional, np.array of shape (3,3), the rotation matrix of the box.
    
    Returns:
    - point_cloud_outside_box: open3d.geometry.PointCloud, the point cloud with points inside the box removed.
    """
    # Create an AxisAlignedBoundingBox or OrientedBoundingBox
    if box_rotation is None:
        bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=box_center - 0.5 * box_extent,
                                                   max_bound=box_center + 0.5 * box_extent)
    else:
        bbox = o3d.geometry.OrientedBoundingBox(center=box_center,
                                                R=box_rotation,
                                                extent=box_extent)
    
    # Get the indices of points inside the box
    indices_inside_box = bbox.get_point_indices_within_bounding_box(point_cloud.points)
    
    # Select points outside the box
    point_cloud_outside_box = point_cloud.select_by_index(indices_inside_box, invert=True)
    
    return point_cloud_outside_box

def gps_to_decimeters(lat1, lon1, lat2, lon2):
    """
    Convert two GPS coordinates to meters relative to the first coordinate.
    
    :param lat1: Latitude of the first coordinate in decimal degrees
    :param lon1: Longitude of the first coordinate in decimal degrees
    :param lat2: Latitude of the second coordinate in decimal degrees
    :param lon2: Longitude of the second coordinate in decimal degrees
    :return: A tuple (x, y) representing the second coordinate in meters relative to the first
    """
    # Radius of the Earth in meters
    R = 6378137.0
    
    # Convert degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Differences in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Equirectangular approximation
    x_meters = dlon * math.cos((lat1_rad + lat2_rad) / 2.0) * R
    y_meters = dlat * R
    
    # Convert to centimeters
    x_decimeters = x_meters * 10
    y_decimeters = y_meters * 10
    
    return x_decimeters, y_decimeters

def mean_filter_column(matrix, window_size):
    """
    Apply a mean filter to each column of a matrix individually.
    
    Parameters:
    matrix (np.ndarray): Input matrix of shape (n, m)
    window_size (int): Size of the moving window for the mean filter
    
    Returns:
    np.ndarray: Matrix with mean filter applied to each column
    """
    if window_size < 1:
        raise ValueError("window_size must be at least 1")
        
    # Create a new matrix to store the results
    filtered_matrix = np.zeros_like(matrix)
    
    # Get the number of rows and columns
    n, m = matrix.shape
    
    # Apply the mean filter to each column
    for col in range(m):
        column = matrix[:, col]
        filtered_column = np.zeros(n)
        
        # Apply the mean filter to the current column
        for row in range(n):
            start_idx = max(0, row - window_size // 2)
            end_idx = min(n, row + window_size // 2 + 1)
            filtered_column[row] = np.mean(column[start_idx:end_idx])
        
        filtered_matrix[:, col] = filtered_column
    
    return filtered_matrix

def vector_to_polar(x, y):
    # Compute the magnitude
    magnitude = math.sqrt(x**2 + y**2)
    
    # Compute the angle in radians
    theta = math.atan2(y, x)  # atan2 returns the angle in radians

    return magnitude, theta

def normalize_angle(angle):
    """
    Normalize an angle to the range [-pi, pi].

    Parameters:
    angle (float): The angle in radians to normalize.

    Returns:
    float: The normalized angle in the range [-pi, pi].
    """
    # Normalize the angle to the range [-pi, pi]
    normalized_angle = (angle + math.pi) % (2 * math.pi) - math.pi
    return normalized_angle

# This function gives a coordinate system with relative movements and accumulated
def coordinated_system(gps_path):
    # It eliminates first element to make the relative movement
    file_gps_locations = open(gps_path, "r")
    # It reads the line twice to get rid of the title.
    prior_element      = file_gps_locations.readline()
    prior_element      = file_gps_locations.readline()
    relative_magnitude = []
    relative_theta     = []
    # This keeps moving the origin and orientation
    absolute_x_matrix  = []
    absolute_x_scalar  = 0
    absolute_y_matrix  = []
    absolute_y_scalar  = 0
    absolute_theta     = []
    accumulated_theta  = 0
    timestamp_matrix   = []
    delta_timestamp    = []
    for line_gps in file_gps_locations:
        timestamp_0        = int(float(prior_element.split(',')[0]))
        latitude_0         = float(prior_element.split(',')[2])
        longitude_0        = float(prior_element.split(',')[3])
        timestamp_1        = int(float(line_gps.split(',')[0]))
        latitude_1         = float(line_gps.split(',')[2])
        longitude_1        = float(line_gps.split(',')[3])
        # Update element
        prior_element      = line_gps
        x, y               = gps_to_decimeters(latitude_0, longitude_0, latitude_1, longitude_1)
        magnitude, theta   = vector_to_polar(x, y)
        # Accumulates relative trajectory.
        relative_magnitude.append(magnitude)
        # The sampling rate is approximately 30 Hz, then if the truck moves not even 1 cm in that period, then, it is still
        if absolute_x_scalar < 0.1 and absolute_y_scalar < 0.1:
            theta = 0
        relative_theta.append(theta)
        # It translates origin and accumulates theta
        absolute_x_scalar  += x
        absolute_y_scalar  += y
        accumulated_theta  += theta
        absolute_x_matrix.append(absolute_x_scalar)
        absolute_y_matrix.append(absolute_y_scalar)
        absolute_theta.append(accumulated_theta)
        timestamp_matrix.append(timestamp_1)
        delta_timestamp.append(timestamp_1 - timestamp_0)
    # It returns matrix with accumulated values
    result_matrix = relative_magnitude
    result_matrix = np.vstack((result_matrix,relative_theta))
    result_matrix = np.vstack((result_matrix,absolute_x_matrix))
    result_matrix = np.vstack((result_matrix,absolute_y_matrix))
    result_matrix = np.vstack((result_matrix,absolute_theta))
    result_matrix = np.vstack((result_matrix,timestamp_matrix))
    result_matrix = np.transpose(result_matrix)
    mean_time     = np.mean(delta_timestamp)
    return result_matrix,mean_time

def fps_sinchronize(coordinate_matrix, video_path_timestamps, fps_video, video_duration_seconds):
    file_video_l = open(video_path_timestamps,"r")
    first_time_v = int(file_video_l.readline())
    # It is used to calculate the required timestamps
    frame_cal_ar = []
    for frame_number in range(video_duration_seconds * fps_video):
        frame_cal_ar.append(first_time_v + frame_number * 1e9 / fps_video)
    magnitude_m  = []
    rel_theta_m  = []
    absolute_x_m = []
    absolute_y_m = []
    absolute_t_m = []
    timestamp_m  = []
    
    log_file_path = 'log_timestamps_pcd.txt'
    if not os.path.exists(log_file_path):
        os.mknod(log_file_path)
        
    file = open(log_file_path,'a')
    # Look for the frame with the lower difference
    for frame_cal in frame_cal_ar:
        # To initialize value
        threshold      = frame_cal
        magnitude      = 0
        relative_theta = 0
        absolute_x     = 0
        absolute_y     = 0
        absolute_theta = 0
        timestamp      = 0
        # Look through all the rows
        for index in range(coordinate_matrix.shape[0]):
            cal_time_stamp = abs(frame_cal - coordinate_matrix[index,5])
            if(cal_time_stamp < threshold):
                magnitude  = coordinate_matrix[index,0]
                r_theta    = coordinate_matrix[index,1]
                absolute_x = coordinate_matrix[index,2]
                absolute_y = coordinate_matrix[index,3]
                absolute_t = coordinate_matrix[index,4]
                timestamp  = coordinate_matrix[index,5]
                threshold  = cal_time_stamp
        # Save the last value with shortest threshold.
        magnitude_m.append(magnitude)
        rel_theta_m.append(r_theta)
        absolute_x_m.append(absolute_x)
        absolute_y_m.append(absolute_y)
        absolute_t_m.append(absolute_t)
        timestamp_m.append(timestamp)
        file.write(str(magnitude) + '/' + str(r_theta) + '/' + str(absolute_x) + '/' + str(absolute_y) + '/' + str(timestamp) + '\n')
        
    result_matrix = magnitude_m
    result_matrix = np.vstack((result_matrix,rel_theta_m))
    result_matrix = np.vstack((result_matrix,absolute_x_m))
    result_matrix = np.vstack((result_matrix,absolute_y_m))
    result_matrix = np.vstack((result_matrix,absolute_t_m))
    result_matrix = np.vstack((result_matrix,timestamp_m))
    result_matrix = np.transpose(result_matrix)
    file.close()
    return result_matrix




def fps_sinchronize_frames(coordinate_matrix, video_path_timestamps, fps_video, video_duration_frames):
    file_video_l = open(video_path_timestamps,"r")
    file_vid_rl  = file_video_l.readline()
    if(not(file_vid_rl.isnumeric())):
        file_vid_rl = file_vid_rl.split('.')[0]
    first_time_v = int(file_vid_rl)
    # It is used to calculate the required timestamps
    frame_cal_ar = []
    for frame_number in range(video_duration_frames):
        frame_cal_ar.append((first_time_v + frame_number / fps_video) * 1e9)
    magnitude_m  = []
    rel_theta_m  = []
    absolute_x_m = []
    absolute_y_m = []
    absolute_t_m = []
    timestamp_m  = []
    
    log_file_path = 'log_timestamps_pcd.txt'
    if not os.path.exists(log_file_path):
        os.mknod(log_file_path)
        
    file = open(log_file_path,'a')
    # Look for the frame with the lower difference
    for frame_cal in frame_cal_ar:
        # To initialize value
        threshold      = frame_cal
        magnitude      = 0
        r_theta        = 0
        absolute_x     = 0
        absolute_y     = 0
        absolute_t     = 0
        absolute_theta = 0
        timestamp      = 0
        # Look through all the rows
        for index in range(coordinate_matrix.shape[0]):
            cal_time_stamp = abs(frame_cal - coordinate_matrix[index,5])
            if(cal_time_stamp < threshold):
                magnitude  = coordinate_matrix[index,0]
                r_theta    = coordinate_matrix[index,1]
                absolute_x = coordinate_matrix[index,2]
                absolute_y = coordinate_matrix[index,3]
                absolute_t = coordinate_matrix[index,4]
                timestamp  = coordinate_matrix[index,5]
                threshold  = cal_time_stamp
        # Save the last value with shortest threshold.
        magnitude_m.append(magnitude)
        rel_theta_m.append(r_theta)
        absolute_x_m.append(absolute_x)
        absolute_y_m.append(absolute_y)
        absolute_t_m.append(absolute_t)
        timestamp_m.append(timestamp)
        file.write(str(magnitude) + '/' + str(r_theta) + '/' + str(absolute_x) + '/' + str(absolute_y) + '/' + str(timestamp) + '\n')
        
    result_matrix = magnitude_m
    result_matrix = np.vstack((result_matrix,rel_theta_m))
    result_matrix = np.vstack((result_matrix,absolute_x_m))
    result_matrix = np.vstack((result_matrix,absolute_y_m))
    result_matrix = np.vstack((result_matrix,absolute_t_m))
    result_matrix = np.vstack((result_matrix,timestamp_m))
    result_matrix = np.transpose(result_matrix)
    file.close()
    return result_matrix

def load_point_clouds_rotated_translated(pcd_front_rear, theta, origin_x, origin_y, voxel_size=0.0):
    #Rotations are relative to front or forward
    rotate_rear            = (np.pi, 0, 0)
    rotate_theta           = (theta, 0, 0)
    # To change magnitude from centimeters to decimeters
    translate_mt           = (0, origin_x, origin_y)
    
    # Calculate first front
    mesh                   = copy.deepcopy(pcd_front_rear[0])
    rotated_theta          = np.eye(4)
    rotated_theta[:3, :3]  = pcd_front_rear[0].get_rotation_matrix_from_xyz(rotate_theta)
    # Move pcd forward a magnitude and then rotated theta radians. In this order to remain forward
    mesh                   = mesh.translate(translate_mt)
    mesh                   = mesh.transform(rotated_theta)
    pcd_down_front         = mesh.voxel_down_sample(voxel_size=voxel_size)
    
    # Calculate then rear
    rotated_matrix         = np.eye(4)
    rotated_matrix[:3, :3] = pcd_front_rear[1].get_rotation_matrix_from_xyz(rotate_rear)
    mesh                   = copy.deepcopy(pcd_front_rear[1]).transform(rotated_matrix)
    rotated_theta          = np.eye(4)
    rotated_theta[:3, :3]  = pcd_front_rear[1].get_rotation_matrix_from_xyz(rotate_theta)
    # Rotate pcd forward and position it x and y decemiters. In this order to remain forward
    mesh                   = mesh.transform(rotated_theta)
    mesh                   = mesh.translate(translate_mt)
    pcd_down_rear          = mesh.voxel_down_sample(voxel_size=voxel_size)
    
    # Define the box parameters
    #center                 = np.array([0.0, 0.0, 225])
    # Consider 70
    #extent                 = np.array([400, 400, 100])
    #rotation               = np.eye(3)  # Identity matrix for no rotation
    # Cut the points inside the box, eliminate point in infinite or far far away
    #pcd_front_outside_box  = cut_points_in_box(pcd_down_front, center, extent, rotation)
    
    #return pcd_down_front + pcd_down_rear
    # For now just collect front, rear has a pipe in the middle that will get noise into the scene
    return pcd_down_front

def accumulate_pcd(pcds):
    pcd_join_pcd = o3d.geometry.PointCloud()
    for current_pcd in pcds:
        pcd_join_pcd += current_pcd
    # Returns PCD recollection 
    return pcd_join_pcd

def map_generation(forward_depth_path, forward_color_path, chunks_reduction, video_duration, fps, forward_file_name_begin, rear_file_name_begin, coordinated_matrix):
    brightness_value   = 0
    voxel_size         = 2
    pcd_chunks_accumul = []

    for part_of_map in range(chunks_reduction):
        # To do it with a blank each time
        pcd_accumulated = []
        chunk_duration  = int(video_duration * fps / chunks_reduction)
        offset          = part_of_map * chunk_duration
        for index in range(chunk_duration):
            print('Processing frame: ' +  str(offset + index))
            pcd_studied       = []
            forward_file_name = forward_file_name_begin + str(offset + index)
            rear_file_name    = rear_file_name_begin + str(offset + index)
            depth_file_name_f = forward_depth_path + forward_file_name + '.png'
            color_file_name_f = forward_color_path + forward_file_name + '.jpg'
            depth_file_name_r = forward_depth_path + forward_file_name + '.png'
            color_file_name_r = forward_color_path + forward_file_name + '.jpg'
            # Accumulate front and then rear for the next function
            theta             = coordinated_matrix[offset + index,1]
            origin_x          = coordinated_matrix[offset + index,2]
            origin_y          = coordinated_matrix[offset + index,3]
            pcd_studied.append(depth_to_pcd(depth_file_name_f, color_file_name_f, brightness_value))
            pcd_studied.append(depth_to_pcd(depth_file_name_r, color_file_name_r, brightness_value))
            pcd_accumulated.append(load_point_clouds_rotated_translated(pcd_studied, theta, origin_x, origin_y, voxel_size))
        pcd_chunks_accumul.append(pcd_accumulated)
    return pcd_chunks_accumul

def map_generation_frames(forward_depth_path, forward_color_path, chunks_reduction, video_duration_frames, fps, forward_file_name_begin, coordinated_matrix):
    brightness_value   = 0
    voxel_size         = 2
    pcd_chunks_accumul = []
    
    num_digits = len(str(video_duration_frames))

    for part_of_map in range(chunks_reduction):
        # To do it with a blank each time
        pcd_accumulated = []
        chunk_duration  = int(video_duration_frames / chunks_reduction)
        offset          = part_of_map * chunk_duration
        for index in range(chunk_duration):
            print('Processing frame: ' +  str(offset + index))
            pcd_studied       = []
            #/kaggle/working/depth/soads_2024-06-26-16-51-42-_front_camera_image_compressed_1309.jpg
            #/kaggle/working/color/soads_2024-06-26-16-51-42-_front_camera_image_compressed325.jpg
            
            formatted_number = f"{(offset + index):0{num_digits}d}"

            print(f"offset: {offset}, index: {index}, num_digits: {num_digits}, formatted_number: {formatted_number}")
            
            # For 1 and 4
            #forward_file_name_d = forward_file_name_begin + str(offset + index)
            forward_file_name_d = forward_file_name_begin + formatted_number
            # For 2 and 3
            #forward_file_name_d = forward_file_name_begin + '_complete' + str(offset + index)
            
            forward_file_name_c = forward_file_name_d
            depth_file_name_f = forward_depth_path + forward_file_name_d + '.png'
            color_file_name_f = forward_color_path + forward_file_name_c + '.png'
            # Accumulate front and then rear for the next function
            theta             = coordinated_matrix[offset + index,1]
            origin_x          = coordinated_matrix[offset + index,2]
            origin_y          = coordinated_matrix[offset + index,3]
            pcd_studied.append(depth_to_pcd(depth_file_name_f, color_file_name_f, brightness_value))
            pcd_studied.append(depth_to_pcd(depth_file_name_f, color_file_name_f, brightness_value))
            pcd_accumulated.append(load_point_clouds_rotated_translated(pcd_studied, theta, origin_x, origin_y, voxel_size))
        pcd_chunks_accumul.append(pcd_accumulated)
    return pcd_chunks_accumul

def frame_extractor(input_video, output_dir):
    video_local = cv2.VideoCapture(input_video)
    os.makedirs(output_dir, exist_ok=True)
    i = 0
    while(video_local.isOpened()):
        ret, frame = video_local.read()
        if ret == False:
            break
        cv2.imwrite(output_dir + input_video.split("/")[-1].split(".")[0] + str(i) + '.jpg',frame)
        i+=1
    video_local.release()
    return i

def synchronize_video_duration(coordinated_matrix,delta_timestamp_gps,fps_video,video_initial_seconds,duration_seconds):
    rounded_fps_ms_gps  = round(1000 / delta_timestamp_gps)
    rounded_fps_ms_vid  = round(fps_video)
    ratio_gps_vid       = math.ceil(rounded_fps_ms_gps / rounded_fps_ms_vid)
    ratio_vid_gps       = math.ceil(rounded_fps_ms_vid / rounded_fps_ms_gps)
    video_coordinates   = []
    
    video_initial_ms    = video_initial_seconds * 1000
    duration_seconds_ms = duration_seconds * 1000
    # We require an int number
    initial_frame_video = round(video_initial_ms / rounded_fps_ms_vid)
    
    final_frame_video   = initial_frame_video + round(duration_seconds_ms / rounded_fps_ms_vid)
    
    video_coordinates = match_gps_to_video(coordinated_gps_matrix, fps_video, fps_gps)
    
    if (rounded_fps_ms_gps < rounded_fps_ms_vid):
        for frame_studied in range(final_frame_video - initial_frame_video):
            for frame_studied_gps in range(ratio_gps_vid):
                #print(coordinated_matrix[(frame_studied + initial_frame_video) * ratio_gps_vid])
                video_coordinates.append(coordinated_matrix[(frame_studied + initial_frame_video) ])
    else:
        for frame_studied in range(final_frame_video - initial_frame_video):
            #print(coordinated_matrix[(frame_studied + initial_frame_video) * ratio_gps_vid])
            video_coordinates.append(coordinated_matrix[(frame_studied + initial_frame_video) * ratio_gps_vid])
    return video_coordinates

def match_gps_to_video(coordinated_gps_matrix, fps_video, fps_gps):
    """
    Adjusts GPS data to match the frame rate of the video.
    If fps_video > fps_gps, it interpolates between points.
    If fps_gps > fps_video, it erodes the GPS matrix by downsampling.

    Parameters:
    coordinated_gps_matrix (ndarray): The GPS data as a 2D array of shape (x, 6).
    fps_video (int): Frame rate of the video.
    fps_gps (int): Frame rate of the GPS data.

    Returns:
    ndarray: Adjusted GPS data matching the video frame rate.
    """

    if fps_video > fps_gps:
        # Case 1: Interpolation (fps_video > fps_gps)
        ratio = fps_video / fps_gps
        interpolated_gps_data = []

        for i in range(len(coordinated_gps_matrix) - 1):
            # Add the original GPS point
            interpolated_gps_data.append(coordinated_gps_matrix[i])

            # Calculate the number of interpolated points between current and next GPS point
            num_interpolated_points = int(ratio) - 1

            # Linearly interpolate between the current and next GPS point
            for j in range(1, num_interpolated_points + 1):
                interpolated_value = coordinated_gps_matrix[i] + (j / (num_interpolated_points + 1)) * (
                    coordinated_gps_matrix[i + 1] - coordinated_gps_matrix[i]
                )
                interpolated_gps_data.append(interpolated_value)

        # Add the last GPS point
        interpolated_gps_data.append(coordinated_gps_matrix[-1])

        return np.array(interpolated_gps_data)

    else:
        # Case 2: Erosion (fps_gps > fps_video)
        ratio = fps_gps / fps_video
        eroded_gps_data = []

        # Downsample the GPS data by selecting every 'ratio'-th point
        for i in range(0, len(coordinated_gps_matrix), int(ratio)):
            eroded_gps_data.append(coordinated_gps_matrix[i])

        # Ensure the last GPS point is included
        if not np.array_equal(eroded_gps_data[-1], coordinated_gps_matrix[-1]):
            eroded_gps_data.append(coordinated_gps_matrix[-1])

        return np.array(eroded_gps_data)

def truncate_gps_data(coordinated_gps_matrix, fps_video, fps_gps, video_initial_seconds, duration_seconds):
    """
    Truncates the GPS data to match a video segment starting from
    video_initial_seconds and lasting duration_seconds.

    Parameters:
    coordinated_gps_matrix (ndarray): The GPS data as a 2D array of shape (x, 6).
    fps_video (int): Frame rate of the video.
    fps_gps (int): Frame rate of the GPS data.
    video_initial_seconds (int): Start time in seconds for truncation.
    duration_seconds (int): Duration in seconds for the segment.

    Returns:
    ndarray: Truncated and adjusted GPS data for the video segment.
    """
    # Adjust GPS data to match the video frame rate
    adjusted_gps_data = match_gps_to_video(coordinated_gps_matrix, fps_video, fps_gps)

    # Calculate the total number of video frames
    total_video_frames = len(adjusted_gps_data)

    # Calculate the starting and ending frame indices
    start_frame = int(video_initial_seconds * fps_video)
    end_frame = start_frame + int(duration_seconds * fps_video)

    # Truncate the adjusted GPS data to the specified segment
    truncated_gps_data = adjusted_gps_data[start_frame:end_frame]

    return truncated_gps_data

In [ ]:
# GPS coordinates where video started
video_initial = 3 * 60 + 48 # 3 minutes and 48 seconds.
duration      = 45          # 45 seconds
#gps_path      = "/kaggle/input/vtti09-04-videos-from-the-road/CV_Work Zone/GX019270_HERO11 Black-GPS9.csv"
gps_path      = "/kaggle/input/gx019270-hero11-black-gps9-csv/GX019270_HERO11 Black-GPS9.csv"

video  = cv2.VideoCapture(video_path)
fps    = video.get(cv2.CAP_PROP_FPS)
window_filter_size    = 160

coordinated_matrix,timestamp = coordinated_system(gps_path)
coordinated_matrix           = mean_filter_column(coordinated_matrix,window_filter_size)
#sinchronized_matrix   = fps_sinchronize_frames(coordinated_matrix, video_path_timestamps, fps, max_frames)
#video_coordinates            = synchronize_video_duration(coordinated_matrix,timestamp,fps,video_initial,duration)

#print(coordinated_matrix)
#print(coordinated_matrix.shape)
#print(timestamp)

In [ ]:
#video_coordinates = synchronize_video_duration(coordinated_matrix,timestamp,fps,video_initial,duration)

# Convertion from timestamp to fps
fps_gps           = 1000 / timestamp

video_coordinates = truncate_gps_data(coordinated_matrix, round(fps), round(fps_gps), video_initial, duration)

#print(video_coordinates.shape)

In [ ]:
import re

def find_highest_frame_number(folder_path):
    highest_number = 0
    pattern = re.compile(r"frame_(\d+)\.png")

    for filename in os.listdir(folder_path):
        match = pattern.match(filename)
        if match:
            number = int(match.group(1))
            if number > highest_number:
                highest_number = number

    return highest_number

chunk_reduction    = 4
forward_color_path = '/kaggle/working/frames/'
forward_depth_path = '/kaggle/working/output/frames/monocular/'
forward_file_name  = 'frame_0'
max_frame          = find_highest_frame_number(forward_color_path)

pcd_chunks_accumul = map_generation_frames(forward_depth_path, forward_color_path, chunk_reduction, max_frame, fps, forward_file_name, coordinated_matrix)

map_chunks = []
for index in range(len(pcd_chunks_accumul)):
    pcd_map_result = accumulate_pcd(pcd_chunks_accumul[index])
    map_chunks.append(pcd_map_result)

pcd_map_result = accumulate_pcd(map_chunks)

for index in range(len(map_chunks)):
    combined_pcd_path = 'camera_map_point_cloud' + str(index) + '.pcd'
    o3d.io.write_point_cloud(combined_pcd_path, map_chunks[index])

result_map_result = 'point_cloud_completed.pcd'
o3d.io.write_point_cloud(result_map_result, pcd_map_result)

In [ ]:
import cv2
import numpy as np
import open3d as o3d
import sys
import math
import struct
import os
import copy
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import imageio.v3 as iio
import matplotlib.pyplot as plt
import open3d as o3d
import math
import copy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import imageio.v3 as iio
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
from numpy import asarray
from skimage.io import imread, imshow
from skimage.color import rgb2hsv

def reduce_point_cloud_density(point_cloud_path, target_size_mb):
    pcd = o3d.io.read_point_cloud(point_cloud_path)
        
    current_size_mb = os.path.getsize(point_cloud_path) / (1024 * 1024)

    # Compute the number of points to keep
    num_points           = len(pcd.points)
    reduction_percentage = (1 - target_size_mb / current_size_mb) * 100
    num_points_to_keep   = int(num_points * (1 - reduction_percentage / 100.0))

    # Create an array of indices and shuffle it
    indices = np.arange(num_points)
    np.random.shuffle(indices)

    # Select the subset of points
    selected_indices = indices[:num_points_to_keep]

    # Downsample the point cloud
    downsampled_point_cloud = pcd.select_by_index(selected_indices)
    
    return downsampled_point_cloud

pcd_path = '/kaggle/working/point_cloud_completed.pcd'
downsampled_pcd = reduce_point_cloud_density(pcd_path, 1)

In [ ]:
def erosion(mask,kernel_size,iter_erosion):
    kernel = np.ones((kernel_size,kernel_size),np.uint8)
    erosion = cv2.erode(mask,kernel,iterations = iter_erosion)
    return erosion

def opening(mask,kernel_size):
    kernel = np.ones((kernel_size,kernel_size),np.uint8)
    return cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

def extract_signs_frames(image_path,image_orig, frame_count, accumulated_rect):
    # .JPG are the segmented images. image_path
    # .JPG are the original images image_orig
    #image      = cv2.imread(image_path)
    #image_p_or = cv2.imread(image_orig)
    image      = Image.open(image_path)
    image      = image.convert('RGB')
    image      = np.array(image)
    image      = image / 255.0
    image_p_or = Image.open(image_orig)
    image_p_or = image_p_or.convert('RGB')
    image_p_or = np.array(image_p_or)
    image_p_or = image_p_or / 255.0
    #print(type(image))
    hsv        = rgb2hsv(image)
    
    # Constraints to identify the color of traffic signs
    #refer to hue channel (in the colorbar)
    #lower_mask      = hsv[:,:,0] > 0.6
    #refer to hue channel (in the colorbar)
    #upper_mask      = hsv[:,:,0] < 0.8 
    #refer to transparency channel (in the colorbar)
    #saturation_mask = hsv[:,:,1] > 0.4
    
    # Iter 1
    #refer to hue channel (in the colorbar)
    #lower_mask      = hsv[:,:,0] > 0.7
    #refer to hue channel (in the colorbar)
    #upper_mask      = hsv[:,:,0] < 0.8 
    #refer to transparency channel (in the colorbar)
    #lower_saturation = hsv[:,:,1] > 0.65
    #refer to transparency channel (in the colorbar)
    #upper_saturation = hsv[:,:,1] < 0.75
    
    # Iter 2
    #refer to hue channel (in the colorbar)
    #lower_mask      = hsv[:,:,0] > 0.6 
    #refer to hue channel (in the colorbar)
    #upper_mask      = hsv[:,:,0] < 0.8
    #refer to transparency channel (in the colorbar)
    #lower_saturation = hsv[:,:,1] > 0.65
    #refer to transparency channel (in the colorbar)
    #upper_saturation = hsv[:,:,1] < 0.9
    
    # 06-28
    #refer to hue channel (in the colorbar)
    lower_mask      = hsv[:,:,0] > 0.6
    #refer to hue channel (in the colorbar)
    upper_mask      = hsv[:,:,0] < 0.8
    #refer to transparency channel (in the colorbar)
    lower_saturation = hsv[:,:,1] > 0.6
    #refer to transparency channel (in the colorbar)
    upper_saturation = hsv[:,:,1] < 0.7
    
    # This is to change the kernel size and eliminate outliers
    #mask              = np.array(upper_mask*lower_mask*saturation_mask,dtype=np.uint8)
    mask              = np.array(upper_mask * lower_mask * upper_saturation * lower_saturation,dtype=np.uint8)
    kernel_size       = 6
    iter_erosion      = 2
    #mask              = erosion(mask,kernel_size,iter_erosion)
    #mask              = opening(mask,kernel_size)
    #plt.imshow(opening_mask)
    #red               = image_p_or[:,:,0]*mask
    #green             = image_p_or[:,:,1]*mask
    #blue              = image_p_or[:,:,2]*mask
    red = (image_p_or[:, :, 2] * mask * 255).astype(np.uint8)
    green = (image_p_or[:, :, 1] * mask * 255).astype(np.uint8)
    blue = (image_p_or[:, :, 0] * mask * 255).astype(np.uint8)
    image_masked      = np.dstack((red,green,blue))
    contours, _       = cv2.findContours(red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    #To avoid outliers
    #min_area          = 500
    min_area          = 1000
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) >= min_area]
    bounding_rects    = [cv2.boundingRect(cnt) for cnt in filtered_contours]
    result_image      = image_masked.copy()
    
    output_dir        = 'cropped_images/'
    os.makedirs(output_dir, exist_ok=True)
    
    print('Frame has ' + str(len(bounding_rects)) + ' rectangles')
    
    log_file_path = output_dir + 'signs_location.txt'
    # If it does not exist, create it
    if not os.path.exists(log_file_path):
        os.mknod(log_file_path)

    # Save the individual cropped images
    for i, rect in enumerate(bounding_rects):
        x, y, w, h    = rect
        cropped_image = image_masked[y:y+h, x:x+w]
        output_path   = os.path.join(output_dir, f'cropped_{accumulated_rect + i}.jpg')
        file          = open(log_file_path,'a')
        file.write(str(frame_count) + '/' + str(accumulated_rect + i) + '\n')
        file.close()
        cv2.imwrite(output_path, cropped_image)
    
    # It tells how many rectangles are there
    return len(bounding_rects)
    
def extract_signs_from_frames(forward_segm_path, forward_color_path, max_frame, forward_file_name_begin):
    rectangles_accum = 0
    
    num_digits = len(str(max_frame))
    
    for index in range(max_frame):
        print('Processing frame: ' +  str(index))
        # For 1 and 4, 2 and 3 too after fix
        forward_file_name_s = forward_file_name_begin + '_1' + str(index)
        # For 2 and 3, this files have an error and when they were merge, fps change from 30 to 25.
        # When fps are different
        #forward_file_name_s = forward_file_name_begin + '_complete' + str(int(index * 25 / 30) + 1)
        
        formatted_number = f"{(index):0{num_digits}d}"
        
        #forward_file_name_c = forward_file_name_begin + str(index)
        
        forward_file_name_c = forward_file_name_begin + formatted_number
        
        color_file_name_f   = forward_color_path + forward_file_name_c + '.png'
        segme_file_name_f   = forward_segm_path + forward_file_name_c + '.png'

        # Get the segmented image with these names
        rectangles_current  = extract_signs_frames(segme_file_name_f,color_file_name_f,index,rectangles_accum)
        rectangles_accum    += rectangles_current
            
def frame_extractor(input_video, output_dir, resize_width, resize_height):
    video_local = cv2.VideoCapture(input_video)
    os.makedirs(output_dir, exist_ok=True)
    i = 0
    while(video_local.isOpened()):
        ret, frame = video_local.read()
        if ret == False:
            break
        #frame = cv2.resize(frame, (resize_width, resize_height))
        cv2.imwrite(output_dir + input_video.split("/")[-1].split(".")[0] + str(i) + '.jpg',frame)
        i+=1
    video_local.release()
    return i

def extract_signs(image_path, image_orig, output_path):
    # Open the images and convert to RGB
    image = Image.open(image_path).convert('RGB')
    image = np.array(image) / 255.0  # Normalize to 0-1 range
    image_p_or = Image.open(image_orig).convert('RGB')
    image_p_or = np.array(image_p_or) / 255.0  # Normalize to 0-1 range

    # Convert to HSV color space
    hsv = rgb2hsv(image)

    # Apply color filtering constraints
    lower_mask = hsv[:, :, 0] > 0.6
    upper_mask = hsv[:, :, 0] < 0.8
    lower_saturation = hsv[:, :, 1] > 0.6
    upper_saturation = hsv[:, :, 1] < 0.7

    # Create the binary mask
    mask = (upper_mask & lower_mask & upper_saturation & lower_saturation).astype(np.uint8)  # Convert to uint8

    # Optional: Apply morphological operations if needed
    kernel_size = 6
    iter_erosion = 2
    # mask = cv2.erode(mask, np.ones((kernel_size, kernel_size), np.uint8), iterations=iter_erosion)
    # mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((kernel_size, kernel_size), np.uint8))

    # Apply the mask to the original image
    red = (image_p_or[:, :, 2] * mask * 255).astype(np.uint8)
    green = (image_p_or[:, :, 1] * mask * 255).astype(np.uint8)
    blue = (image_p_or[:, :, 0] * mask * 255).astype(np.uint8)
    image_masked = np.dstack((red, green, blue))

    # Save the masked image
    cv2.imwrite(output_path, image_masked)

    # Use only one channel (e.g., red channel) for contour detection
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # Filter out small contours
    min_area = 1000
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) >= min_area]

    # Get bounding rectangles for the filtered contours
    bounding_rects = [cv2.boundingRect(cnt) for cnt in filtered_contours]

    return len(bounding_rects)


def increase_brightness(image_path, value):
    # Read the image
    image = cv2.imread(image_path)
    # Check if image is loaded successfully
    if image is None:
        raise ValueError(f"Image at path '{image_path}' could not be loaded.")
    # Convert the image to HSV (Hue, Saturation, Value) color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Split the image into three channels: H, S, and V
    h, s, v = cv2.split(hsv)
    # Increase the V (value/brightness) channel by the given value
    v = cv2.add(v, value)
    # Merge the channels back
    final_hsv = cv2.merge((h, s, v))
    # Convert the HSV image back to BGR color space
    brightened_image = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return brightened_image

#This is for Vision Transformers because it gives an inverse relation of white and black is.
def normalize_image_to_pcd(input_path,brightness_value):
    brightness_image  = increase_brightness(input_path,brightness_value)
    cv2.imwrite('brightened_image.jpg', brightness_image)
    input_image       = Image.open('brightened_image.jpg').convert('L')
    input_image_array = input_image.resize((640,480))
    # It reverse depth making black, white and viceversa.
    #max_pixel_value   = input_image_array
    reverse_depth     = 255 - asarray(input_image_array)
    #print(len(reverse_depth))
    return(reverse_depth)

#It extracts color from image.
def extract_color_image(img_path):
    img = Image.open(img_path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    if img.size != (640,480):
        img = img.resize((640, 480))
    img = asarray(img)
    img = img/255.0 # normalize RGB values to [0, 1]
    height, width, channels = img.shape
    length = height * width

    color_list = img.reshape((length, channels)) # array of RGB values
    return color_list

#It creates the PCD from depth.
def depth_to_pcd(depth_path, color_image_path, brightness_value, rectangles_found):
    if (rectangles_found == 0):
        pcd_o3d   = o3d.geometry.PointCloud()
        exist_pcd = False
    else:
        # Depth camera parameters from NYU:
        FX_DEPTH    = 5.8262448167737955e+02
        FY_DEPTH    = 5.8269103270988637e+02
        CX_DEPTH    = 3.1304475870804731e+02
        CY_DEPTH    = 2.3844389626620386e+02
        exist_pcd   = True
        # It only has one channel it is in grey scale, it change size and invert greyscale
        depth_image = normalize_image_to_pcd(depth_path,brightness_value)
        # get depth resolution:
        height, width = depth_image.shape
        length = height * width
        # compute indices:
        jj = np.tile(range(width), height)
        ii = np.repeat(range(height), width)
        # rechape depth image
        z = depth_image.reshape(length)
        # compute pcd:
        pcd = np.dstack([(ii - CX_DEPTH) * z / FX_DEPTH,
                     (jj - CY_DEPTH) * z / FY_DEPTH,
                     z]).reshape((length, 3))

        pcd_o3d = o3d.geometry.PointCloud()
        pcd_o3d.points = o3d.utility.Vector3dVector(pcd)
        colors = extract_color_image(color_image_path)
        pcd_o3d.colors = o3d.utility.Vector3dVector(colors)
        #o3d.io.write_point_cloud(output_pcd_path, pcd_o3d)

        # Define the box parameters
        center   = np.array([0.0, 0.0, 250])
        # Consider 70
        extent   = np.array([400, 400, 90])
        rotation = np.eye(3)  # Identity matrix for no rotation
        # Cut the points inside the box, eliminate point in infinite or far far away
        pcd_o3d  = cut_points_in_box(pcd_o3d, center, extent, rotation)
    return pcd_o3d,exist_pcd

def cut_points_in_box(point_cloud, box_center, box_extent, box_rotation=None):
    """
    Cut points inside a box from the point cloud.
    
    Parameters:
    - point_cloud: open3d.geometry.PointCloud, the input point cloud.
    - box_center: list or np.array of shape (3,), the center of the box.
    - box_extent: list or np.array of shape (3,), the size of the box (x, y, z dimensions).
    - box_rotation: optional, np.array of shape (3,3), the rotation matrix of the box.
    
    Returns:
    - point_cloud_outside_box: open3d.geometry.PointCloud, the point cloud with points inside the box removed.
    """
    
    #points          = np.asarray(point_cloud.points)
    #z_threshold     = 188
    #mask            = points[:,2] < z_threshold    
    #point_cloud.points = o3d.utility.Vector3dVector(points[mask]) # normals and colors are unchanged
    
    #return point_cloud
    
    # Create an AxisAlignedBoundingBox or OrientedBoundingBox
    if box_rotation is None:
        bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=box_center - 0.5 * box_extent,
                                                   max_bound=box_center + 0.5 * box_extent)
    else:
        bbox = o3d.geometry.OrientedBoundingBox(center=box_center,
                                                R=box_rotation,
                                                extent=box_extent)
    
    # Get the indices of points inside the box
    indices_inside_box = bbox.get_point_indices_within_bounding_box(point_cloud.points)
    
    # Select points outside the box
    point_cloud_outside_box = point_cloud.select_by_index(indices_inside_box, invert=True)
    
    return point_cloud_outside_box

def gps_to_decimeters(lat1, lon1, lat2, lon2):
    """
    Convert two GPS coordinates to meters relative to the first coordinate.
    
    :param lat1: Latitude of the first coordinate in decimal degrees
    :param lon1: Longitude of the first coordinate in decimal degrees
    :param lat2: Latitude of the second coordinate in decimal degrees
    :param lon2: Longitude of the second coordinate in decimal degrees
    :return: A tuple (x, y) representing the second coordinate in meters relative to the first
    """
    # Radius of the Earth in meters
    R = 6378137.0
    
    # Convert degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Differences in coordinates
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Equirectangular approximation
    x_meters = dlon * math.cos((lat1_rad + lat2_rad) / 2.0) * R
    y_meters = dlat * R
    
    # Convert to centimeters
    x_decimeters = x_meters * 10
    y_decimeters = y_meters * 10
    
    return x_decimeters, y_decimeters

def mean_filter_column(matrix, window_size):
    """
    Apply a mean filter to each column of a matrix individually.
    
    Parameters:
    matrix (np.ndarray): Input matrix of shape (n, m)
    window_size (int): Size of the moving window for the mean filter
    
    Returns:
    np.ndarray: Matrix with mean filter applied to each column
    """
    if window_size < 1:
        raise ValueError("window_size must be at least 1")
        
    # Create a new matrix to store the results
    filtered_matrix = np.zeros_like(matrix)
    
    # Get the number of rows and columns
    n, m = matrix.shape
    
    # Apply the mean filter to each column
    for col in range(m):
        column = matrix[:, col]
        filtered_column = np.zeros(n)
        
        # Apply the mean filter to the current column
        for row in range(n):
            start_idx = max(0, row - window_size // 2)
            end_idx = min(n, row + window_size // 2 + 1)
            filtered_column[row] = np.mean(column[start_idx:end_idx])
        
        filtered_matrix[:, col] = filtered_column
    
    return filtered_matrix

def vector_to_polar(x, y):
    # Compute the magnitude
    magnitude = math.sqrt(x**2 + y**2)
    
    # Compute the angle in radians
    theta = math.atan2(y, x)  # atan2 returns the angle in radians

    return magnitude, theta

def normalize_angle(angle):
    """
    Normalize an angle to the range [-pi, pi].

    Parameters:
    angle (float): The angle in radians to normalize.

    Returns:
    float: The normalized angle in the range [-pi, pi].
    """
    # Normalize the angle to the range [-pi, pi]
    normalized_angle = (angle + math.pi) % (2 * math.pi) - math.pi
    return normalized_angle

# This function gives a coordinate system with relative movements and accumulated
def coordinated_system(gps_path):
    # It eliminates first element to make the relative movement
    file_gps_locations = open(gps_path, "r")
    prior_element      = file_gps_locations.readline()
    relative_magnitude = []
    relative_theta     = []
    # This keeps moving the origin and orientation
    absolute_x_matrix  = []
    absolute_x_scalar  = 0
    absolute_y_matrix  = []
    absolute_y_scalar  = 0
    absolute_theta     = []
    accumulated_theta  = 0
    timestamp_matrix   = []
    for line_gps in file_gps_locations:
        timestamp_0        = int(prior_element.split('/')[0])
        latitude_0         = float(prior_element.split('/')[1])
        longitude_0        = float(prior_element.split('/')[2])
        timestamp_1        = int(line_gps.split('/')[0])
        latitude_1         = float(line_gps.split('/')[1])
        longitude_1        = float(line_gps.split('/')[2])
        # Update element
        prior_element      = line_gps
        x, y               = gps_to_decimeters(latitude_0, longitude_0, latitude_1, longitude_1)
        magnitude, theta   = vector_to_polar(x, y)
        # Accumulates relative trajectory.
        relative_magnitude.append(magnitude)
        # The sampling rate is approximately 30 Hz, then if the truck moves not even 1 cm in that period, then, it is still
        if absolute_x_scalar < 0.1 and absolute_y_scalar < 0.1:
            theta = 0
        relative_theta.append(theta)
        # It translates origin and accumulates theta
        absolute_x_scalar  += x
        absolute_y_scalar  += y
        accumulated_theta  += theta
        absolute_x_matrix.append(absolute_x_scalar)
        absolute_y_matrix.append(absolute_y_scalar)
        absolute_theta.append(accumulated_theta)
        timestamp_matrix.append(timestamp_1)
    # It returns matrix with accumulated values
    result_matrix = relative_magnitude
    result_matrix = np.vstack((result_matrix,relative_theta))
    result_matrix = np.vstack((result_matrix,absolute_x_matrix))
    result_matrix = np.vstack((result_matrix,absolute_y_matrix))
    result_matrix = np.vstack((result_matrix,absolute_theta))
    result_matrix = np.vstack((result_matrix,timestamp_matrix))
    result_matrix = np.transpose(result_matrix)
    return result_matrix

def fps_sinchronize(coordinate_matrix, video_path_timestamps, fps_video, video_duration_seconds):
    file_video_l = open(video_path_timestamps,"r")
    first_time_v = int(file_video_l.readline())
    # It is used to calculate the required timestamps
    frame_cal_ar = []
    for frame_number in range(video_duration_seconds * fps_video):
        frame_cal_ar.append(first_time_v + frame_number * 1e9 / fps_video)
    magnitude_m  = []
    rel_theta_m  = []
    absolute_x_m = []
    absolute_y_m = []
    absolute_t_m = []
    timestamp_m  = []
    
    log_file_path = 'log_timestamps_pcd.txt'
    if not os.path.exists(log_file_path):
        os.mknod(log_file_path)
        
    file = open(log_file_path,'a')
    # Look for the frame with the lower difference
    for frame_cal in frame_cal_ar:
        # To initialize value
        threshold      = frame_cal
        magnitude      = 0
        relative_theta = 0
        absolute_x     = 0
        absolute_y     = 0
        absolute_theta = 0
        timestamp      = 0
        # Look through all the rows
        for index in range(coordinate_matrix.shape[0]):
            cal_time_stamp = abs(frame_cal - coordinate_matrix[index,5])
            if(cal_time_stamp < threshold):
                magnitude  = coordinate_matrix[index,0]
                r_theta    = coordinate_matrix[index,1]
                absolute_x = coordinate_matrix[index,2]
                absolute_y = coordinate_matrix[index,3]
                absolute_t = coordinate_matrix[index,4]
                timestamp  = coordinate_matrix[index,5]
                threshold  = cal_time_stamp
        # Save the last value with shortest threshold.
        magnitude_m.append(magnitude)
        rel_theta_m.append(r_theta)
        absolute_x_m.append(absolute_x)
        absolute_y_m.append(absolute_y)
        absolute_t_m.append(absolute_t)
        timestamp_m.append(timestamp)
        file.write(str(magnitude) + '/' + str(r_theta) + '/' + str(absolute_x) + '/' + str(absolute_y) + '/' + str(timestamp) + '\n')
        
    result_matrix = magnitude_m
    result_matrix = np.vstack((result_matrix,rel_theta_m))
    result_matrix = np.vstack((result_matrix,absolute_x_m))
    result_matrix = np.vstack((result_matrix,absolute_y_m))
    result_matrix = np.vstack((result_matrix,absolute_t_m))
    result_matrix = np.vstack((result_matrix,timestamp_m))
    result_matrix = np.transpose(result_matrix)
    file.close()
    return result_matrix

def fps_sinchronize_frames(coordinate_matrix, video_path_timestamps, fps_video, video_duration_frames):
    file_video_l = open(video_path_timestamps,"r")
    file_vid_rl  = file_video_l.readline()
    if(not(file_vid_rl.isnumeric())):
        file_vid_rl = file_vid_rl.split('.')[0]
    first_time_v = int(file_vid_rl)
    # It is used to calculate the required timestamps
    frame_cal_ar = []
    for frame_number in range(video_duration_frames):
        frame_cal_ar.append((first_time_v + frame_number / fps_video) * 1e9)
    magnitude_m  = []
    rel_theta_m  = []
    absolute_x_m = []
    absolute_y_m = []
    absolute_t_m = []
    timestamp_m  = []
    
    log_file_path = 'log_timestamps_pcd.txt'
    if not os.path.exists(log_file_path):
        os.mknod(log_file_path)
        
    file = open(log_file_path,'a')
    # Look for the frame with the lower difference
    for frame_cal in frame_cal_ar:
        # To initialize value
        threshold      = frame_cal
        magnitude      = 0
        r_theta        = 0
        absolute_x     = 0
        absolute_y     = 0
        absolute_t     = 0
        absolute_theta = 0
        timestamp      = 0
        # Look through all the rows
        for index in range(coordinate_matrix.shape[0]):
            cal_time_stamp = abs(frame_cal - coordinate_matrix[index,5])
            if(cal_time_stamp < threshold):
                magnitude  = coordinate_matrix[index,0]
                r_theta    = coordinate_matrix[index,1]
                absolute_x = coordinate_matrix[index,2]
                absolute_y = coordinate_matrix[index,3]
                absolute_t = coordinate_matrix[index,4]
                timestamp  = coordinate_matrix[index,5]
                threshold  = cal_time_stamp
        # Save the last value with shortest threshold.
        magnitude_m.append(magnitude)
        rel_theta_m.append(r_theta)
        absolute_x_m.append(absolute_x)
        absolute_y_m.append(absolute_y)
        absolute_t_m.append(absolute_t)
        timestamp_m.append(timestamp)
        file.write(str(magnitude) + '/' + str(r_theta) + '/' + str(absolute_x) + '/' + str(absolute_y) + '/' + str(timestamp) + '\n')
        
    result_matrix = magnitude_m
    result_matrix = np.vstack((result_matrix,rel_theta_m))
    result_matrix = np.vstack((result_matrix,absolute_x_m))
    result_matrix = np.vstack((result_matrix,absolute_y_m))
    result_matrix = np.vstack((result_matrix,absolute_t_m))
    result_matrix = np.vstack((result_matrix,timestamp_m))
    result_matrix = np.transpose(result_matrix)
    file.close()
    return result_matrix

def load_point_clouds_rotated_translated(pcd_front_rear, theta, origin_x, origin_y, voxel_size=0.0):
    #Rotations are relative to front or forward
    rotate_rear            = (np.pi, 0, 0)
    rotate_theta           = (theta, 0, 0)
    # To change magnitude from centimeters to decimeters
    translate_mt           = (0, origin_x, origin_y)
    
    # Calculate first front
    mesh                   = copy.deepcopy(pcd_front_rear)
    rotated_theta          = np.eye(4)
    rotated_theta[:3, :3]  = pcd_front_rear.get_rotation_matrix_from_xyz(rotate_theta)
    # Move pcd forward a magnitude and then rotated theta radians. In this order to remain forward
    mesh                   = mesh.translate(translate_mt)
    mesh                   = mesh.transform(rotated_theta)
    pcd_down_front         = mesh.voxel_down_sample(voxel_size=voxel_size)
    
    # Calculate then rear
    #rotated_matrix         = np.eye(4)
    #rotated_matrix[:3, :3] = pcd_front_rear[1].get_rotation_matrix_from_xyz(rotate_rear)
    #mesh                   = copy.deepcopy(pcd_front_rear[1]).transform(rotated_matrix)
    #rotated_theta          = np.eye(4)
    #rotated_theta[:3, :3]  = pcd_front_rear[1].get_rotation_matrix_from_xyz(rotate_theta)
    # Rotate pcd forward and position it x and y decemiters. In this order to remain forward
    #mesh                   = mesh.transform(rotated_theta)
    #mesh                   = mesh.translate(translate_mt)
    #pcd_down_rear          = mesh.voxel_down_sample(voxel_size=voxel_size)
    
    #return pcd_down_front + pcd_down_rear
    # For now just collect front, rear has a pipe in the middle that will get noise into the scene
    return pcd_down_front

def accumulate_pcd(pcds):
    pcd_join_pcd = o3d.geometry.PointCloud()
    for current_pcd in pcds:
        pcd_join_pcd += current_pcd
    # Returns PCD recollection 
    return pcd_join_pcd

def map_generation(forward_depth_path, forward_color_path, forward_segm_path, chunks_reduction, video_duration, fps, forward_file_name_begin, rear_file_name_begin, coordinated_matrix):
    brightness_value   = 0
    voxel_size         = 2
    pcd_chunks_accumul = []

    for part_of_map in range(chunks_reduction):
        # To do it with a blank each time
        pcd_accumulated = []
        chunk_duration  = int(video_duration * fps / chunks_reduction)
        offset          = part_of_map * chunk_duration
        for index in range(chunk_duration):
            print('Processing frame: ' +  str(offset + index))
            pcd_studied             = []
            forward_file_name       = forward_file_name_begin + str(offset + index)
            rear_file_name          = rear_file_name_begin + str(offset + index)
            depth_file_name_f       = forward_depth_path + forward_file_name + '.png'
            color_file_name_f       = forward_color_path + forward_file_name + '.jpg'
            segme_file_name_f       = forward_segm_path + forward_file_name + '.png'
            depth_file_name_r       = forward_depth_path + forward_file_name + '.png'
            color_file_name_r       = forward_color_path + forward_file_name + '.jpg'
            segme_file_name_r       = forward_segm_path + forward_file_name + '.png'
            # Get the segmented image with these names
            segment_depth_out       = 'segmentation_depth.png'
            rectangles_found        = extract_signs(segme_file_name_f,depth_file_name_f,segment_depth_out)
            segment_color_out       = 'segmentation_color.png'
            rectangles_found        = extract_signs(segme_file_name_f,color_file_name_f,segment_color_out)
            # Accumulate front and then rear for the next function
            theta                   = coordinated_matrix[offset + index,1]
            origin_x                = coordinated_matrix[offset + index,2]
            origin_y                = coordinated_matrix[offset + index,3]
            generated_pcd,exist_pcd = depth_to_pcd(segment_depth_out, segment_color_out, brightness_value, rectangles_found)
            if(exist_pcd):
                pcd_studied.append(generated_pcd)
                pcd_accumulated.append(load_point_clouds_rotated_translated(generated_pcd, theta, origin_x, origin_y, voxel_size))
        pcd_chunks_accumul.append(pcd_accumulated)
    return pcd_chunks_accumul

def map_generation_frames_segmentation(forward_depth_path, forward_color_path, forward_segm_path, chunks_reduction, video_duration_frames, fps, forward_file_name_begin, coordinated_matrix):
    brightness_value   = 0
    voxel_size         = 2
    pcd_chunks_accumul = []
    
    num_digits = len(str(video_duration_frames))

    for part_of_map in range(chunks_reduction):
        # To do it with a blank each time
        pcd_accumulated = []
        chunk_duration  = int(video_duration_frames / chunks_reduction)
        offset          = part_of_map * chunk_duration
        for index in range(chunk_duration):
            print('Processing frame: ' +  str(offset + index))
            pcd_studied       = []
            
            # For 1 and 4
            #forward_file_name_d = forward_file_name_begin + '_1' + str(offset + index)
            # For 2 and 3
            # When fps are different
            #forward_file_name_d = forward_file_name_begin + '_complete' + str(int(index * 25 / 30) + 1)
            #forward_file_name_d = forward_file_name_begin + '_complete' + str(index)
            
            formatted_number = f"{(offset + index):0{num_digits}d}"
            
            forward_file_name_d = forward_file_name_begin + formatted_number
            # For 2 and 3
            #forward_file_name_d = forward_file_name_begin + '_complete' + str(offset + index)
            
            forward_file_name_c = forward_file_name_d
            depth_file_name_f   = forward_depth_path + forward_file_name_d + '.png'
            color_file_name_f   = forward_color_path + forward_file_name_c + '.png'
            segme_file_name_f   = forward_segm_path + forward_file_name_d + '.png'
            
            #After fixing fps from 2 and 3
            #forward_file_name_d = forward_file_name_begin + '_1' + str(offset + index)
            
            #forward_file_name_c = forward_file_name_begin + str(offset + index)
            
            #depth_file_name_f   = forward_depth_path + forward_file_name_d + '.jpg'
            #color_file_name_f   = forward_color_path + forward_file_name_c + '.jpg'
            #segme_file_name_f   = forward_segm_path + forward_file_name_d + '.jpg'
            
            # Get the segmented image with these names
            segment_depth_out       = 'segmentation_depth.png'
            rectangles_found        = extract_signs(segme_file_name_f,depth_file_name_f,segment_depth_out)
            segment_color_out       = 'segmentation_color.png'
            rectangles_found        = extract_signs(segme_file_name_f,color_file_name_f,segment_color_out)
            
            # Accumulate front and then rear for the next function
            theta                   = coordinated_matrix[offset + index,1]
            origin_x                = coordinated_matrix[offset + index,2]
            origin_y                = coordinated_matrix[offset + index,3]
            generated_pcd,exist_pcd = depth_to_pcd(segment_depth_out, segment_color_out, brightness_value, rectangles_found)
            if(exist_pcd):
                pcd_studied.append(generated_pcd)
                pcd_accumulated.append(load_point_clouds_rotated_translated(generated_pcd, theta, origin_x, origin_y, voxel_size))
        pcd_chunks_accumul.append(pcd_accumulated)
    return pcd_chunks_accumul

def frame_extractor(input_video, output_dir, resize_width, resize_height):
    video_local = cv2.VideoCapture(input_video)
    os.makedirs(output_dir, exist_ok=True)
    i = 0
    while(video_local.isOpened()):
        ret, frame = video_local.read()
        if ret == False:
            break
        #frame = cv2.resize(frame, (resize_width, resize_height))
        cv2.imwrite(output_dir + input_video.split("/")[-1].split(".")[0] + str(i) + '.jpg',frame)
        i+=1
    video_local.release()
    return i

In [ ]:
forward_segm_path = '/kaggle/working/output/frames/segmentation/'

pcd_chunks_accumul = map_generation_frames_segmentation(forward_depth_path, forward_color_path, forward_segm_path, chunk_reduction, max_frame, fps, forward_file_name, coordinated_matrix)

map_chunks = []
for index in range(len(pcd_chunks_accumul)):
    pcd_map_result = accumulate_pcd(pcd_chunks_accumul[index])
    map_chunks.append(pcd_map_result)

pcd_map_result = accumulate_pcd(map_chunks)

for index in range(len(map_chunks)):
    combined_pcd_path = 'camera_map_point_cloud' + str(index) + '.pcd'
    o3d.io.write_point_cloud(combined_pcd_path, map_chunks[index])

result_map_result = 'point_cloud_completed_segmented.pcd'
o3d.io.write_point_cloud(result_map_result, pcd_map_result)

In [ ]:
def delete_folder_with_contents(folder_path):
    """
    Deletes a specific folder and all of its contents.

    Parameters:
    folder_path (str): The full path to the folder you want to delete.
    """
    if os.path.exists(folder_path):
        try:
            # Use shutil.rmtree to remove the folder and all of its contents
            shutil.rmtree(folder_path)
            print(f"Successfully deleted folder: {folder_path}")
        except Exception as e:
            print(f"Error deleting folder {folder_path}: {e}")
    else:
        print(f"Folder not found: {folder_path}")
        
delete_folder_with_contents('/kaggle/working/DPT/')

extract_signs_from_frames(forward_segm_path, forward_color_path, max_frame, 'frame_')

In [ ]:
#Writes coordinates
folder_path = '/kaggle/working/'

def write_matrix_shape_to_txt(matrix, filename):
    # Write the matrix to a text file
    np.savetxt(filename, matrix, fmt='%f')

gps_coordinates = 'cropped_images/gps_coordinates_each_line_frame.txt'
write_matrix_shape_to_txt(coordinated_matrix, gps_coordinates)

In [ ]:
import os
import shutil

def move_files_to_folder(files, destination_folder):
    """
    Moves a list of files to a specified folder.

    Parameters:
    files (list of str): List of file paths to be moved.
    destination_folder (str): Path to the destination folder.
    """
    # Check if the destination folder exists, create it if it doesn't
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
        print(f"Created folder: {destination_folder}")

    for file in files:
        if os.path.exists(file):
            try:
                shutil.move(file, destination_folder)
                print(f"Moved file: {file} to {destination_folder}")
            except Exception as e:
                print(f"Error moving file {file}: {e}")
        else:
            print(f"File not found: {file}")
            
def move_folders_to_folder(folders, destination_folder):
    """
    Moves a list of folders and all their contents to a specified folder.

    Parameters:
    folders (list of str): List of folder paths to be moved.
    destination_folder (str): Path to the destination folder.
    """
    # Check if the destination folder exists, create it if it doesn't
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
        print(f"Created folder: {destination_folder}")

    for folder in folders:
        if os.path.exists(folder):
            try:
                # Construct the new path in the destination folder
                folder_name = os.path.basename(folder)
                new_folder_path = os.path.join(destination_folder, folder_name)

                # Move the folder
                shutil.move(folder, new_folder_path)
                print(f"Moved folder: {folder} to {new_folder_path}")
            except Exception as e:
                print(f"Error moving folder {folder}: {e}")
        else:
            print(f"Folder not found: {folder}")
            
files_to_move = ["/kaggle/working/point_cloud_completed.pcd", "/kaggle/working/point_cloud_completed_segmented.pcd"]
destination = "/kaggle/working/work"
move_files_to_folder(files_to_move, destination)

folders_to_move = ["/kaggle/working/cropped_images/"]
move_folders_to_folder(folders_to_move, destination)

In [ ]:
folder_path = '/kaggle/working/'

def delete_all_except(folder_path, exception_folder):
    """
    Deletes all contents in the specified folder except a specific subfolder and all files within that subfolder.

    Parameters:
    folder_path (str): The path to the main folder.
    exception_folder (str): The name of the subfolder to keep (not the full path, just the folder name).
    """
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return

    # Construct the full path to the exception folder
    exception_folder_path = os.path.join(folder_path, exception_folder)
    
    # Check if the exception folder exists
    if not os.path.exists(exception_folder_path) or not os.path.isdir(exception_folder_path):
        print(f"Exception folder not found or is not a directory: {exception_folder_path}")
        return

    # List all items in the main folder
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        
        # Skip the exception folder
        if item_path == exception_folder_path:
            print(f"Skipped folder: {item_path}")
            continue

        # Delete the item (file or folder)
        try:
            if os.path.isfile(item_path):
                os.remove(item_path)
                print(f"Deleted file: {item_path}")
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
                print(f"Deleted folder: {item_path}")
        except Exception as e:
            print(f"Error deleting {item_path}: {e}")
            
delete_all_except(folder_path, destination)

In [ ]:
import os
os.chdir(r'/kaggle/working')

!tar -czf combined.tar.gz *

from IPython.display import FileLink

FileLink(r'combined.tar.gz')